In [0]:
#@title Load Data
# training data
!wget 'https://comp0053-emopain.s3.eu-west-2.amazonaws.com/movementData/train.zip'
!unzip train.zip
# validation data
!wget 'https://comp0053-emopain.s3.eu-west-2.amazonaws.com/movementData/validation.zip'
!unzip validation.zip

import os
import numpy as np
import pandas as pd
from scipy.io import loadmat

exclude_files = ['C382N.mat', 'C93D.mat', 'C544D.mat', 'C788N.mat', 'C382D.mat', 'C56D.mat', 'C709N.mat']
samples = []
for name in os.listdir('./train'):
  if name not in exclude_files:
    sample = loadmat(os.path.join('./train', name))['BD']
    samples.append(sample)

truncated_samples = []
for sample in samples:
  protective_label = sample[:,-1]
  all_indices = np.where(protective_label[:-1] != protective_label[1:])[0]
  padding = 10
  first = all_indices[0] - padding
  last = all_indices[-1] + padding
  truncated_sample = sample[first:last+1, :]
  truncated_samples.append(truncated_sample)

train_data = np.concatenate(truncated_samples)
print("Positive label: ", sum(train_data[:,-1]))
print("Total :", train_data.shape[0])
print("Imbalance: ", sum(train_data[:,-1]) / train_data.shape[0])

val_data  = [loadmat(os.path.join('./validation', name))['BD'] for name in os.listdir('./validation')]
val_data = np.concatenate(val_data)

print("Loaded training data as 'train_data'")
print("Train shape: ", train_data.shape)

print("Loaded training data as 'val_data'")
print("Validation shape: ", val_data.shape)


--2020-05-15 17:54:54--  https://comp0053-emopain.s3.eu-west-2.amazonaws.com/movementData/train.zip
Resolving comp0053-emopain.s3.eu-west-2.amazonaws.com (comp0053-emopain.s3.eu-west-2.amazonaws.com)... 52.95.149.34
Connecting to comp0053-emopain.s3.eu-west-2.amazonaws.com (comp0053-emopain.s3.eu-west-2.amazonaws.com)|52.95.149.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65112138 (62M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  62.10M  23.2MB/s    in 2.7s    

2020-05-15 17:54:57 (23.2 MB/s) - ‘train.zip’ saved [65112138/65112138]

Archive:  train.zip
 extracting: train/C382D.mat         
 extracting: train/C382N.mat         
 extracting: train/C544D.mat         
 extracting: train/C56D.mat          
 extracting: train/C709N.mat         
 extracting: train/C788N.mat         
 extracting: train/C93D.mat          
 extracting: train/P113D.mat         
 extracting: train/P113N.mat         
 extracting: train/P

In [0]:
#@title Load Training Code

import keras
from sklearn.metrics import f1_score
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Input, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score
import random
from sklearn.utils import shuffle
from collections import Counter

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

import pandas as pd


def ceil(a,b):
    return -(-a//b)


def _shape_changing_v3 (x_tr, y_tr, x_va, y_va): 
    window_size = 180
    offset=45
    y_tr, y_va = np.array(y_tr), np.array(y_va)
    x_tr = pd.DataFrame(data=x_tr)     
    x_va = pd.DataFrame(data=x_va)
    new_train_x = []
    new_train_y = []
    for i in range(int(x_tr.shape[0]/offset)-window_size):
        temp_x = x_tr.iloc[i*offset: i*offset + window_size,:]
        y =  Counter(y_tr[i*offset: i*offset + window_size]).most_common(1)
        temp_y = y[0][0]
        new_train_x.append(temp_x.to_numpy())
        new_train_y.append(temp_y)
    new_valid_x = []
    new_valid_y = []
    for i in range(int(x_va.shape[0]/offset)-window_size):
      temp_x = x_va.iloc[i*offset: i*offset + window_size,:]
      y =  Counter(y_va[i*offset: i*offset + window_size]).most_common(1)
      temp_y = y[0][0]
      new_valid_x.append(temp_x.to_numpy())
      new_valid_y.append(temp_y)
    new_train_x = np.array(new_train_x)
    new_train_y = np.array(new_train_y)
    new_valid_x = np.array(new_valid_x)
    new_valid_y = np.array(new_valid_y)
    print(new_train_x.shape, new_train_y.shape)
    print(new_valid_x.shape, new_valid_y.shape)
    return (new_train_x, new_train_y), (new_valid_x, new_valid_y)


num_classes = 2

def custom_training_func(input_tx, input_ty, input_vx, input_vy, epoch):
    train_x, train_y = input_tx, input_ty
    valid_x, valid_y = input_vx, input_vy

    train_y = keras.utils.to_categorical(train_y, num_classes)
    valid_y = keras.utils.to_categorical(valid_y, num_classes)

    # model = Sequential()
    # model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
    # model.add(Dropout(0.5))
    # model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
    # model.add(Dropout(0.5))
    # model.add(Bidirectional(LSTM(units=32)))
    # model.add(Dropout(0.5))
    # model.add(Dense(2, activation='sigmoid'))
    # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model = Sequential()
    model.add(LSTM(units=32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(units=32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(units=32))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', verbose=1)

    checkpoint_path = "stack_bilstm.h5"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1, save_best_only=True, mode='auto')
    callbacks_list = [checkpoint, es]
    history = model.fit(train_x, train_y, validation_split=0.2 , epochs=epoch, callbacks=callbacks_list)  # starts training #validation=(valid_x, valid_y)

    return model, history

from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score

def main (train_x_pre, train_y_pre, valid_x_pre, valid_y_pre, num=1, epochs=10):
  accuracy, f1_avg, f1_score_p, f1_score_n, acc_p, acc_n = 0,0,0,0,0,0
  for i in range(num):
      print(f'================RUN {i}=====================\n')
      # (train_x, train_y), (valid_x, valid_y) = _shape_changing_v2_new(train_x_pre, train_y_pre, valid_x_pre, valid_y_pre)
      (train_x, train_y), (valid_x, valid_y) = _shape_changing_v3(train_x_pre, train_y_pre, valid_x_pre, valid_y_pre)
      # print(train_x.shape, train_y.shape)
      # print(valid_x.shape, valid_y.shape)
      model, history = custom_training_func(train_x, train_y, valid_x, valid_y, epoch=epochs)
      evaluate_y = to_categorical(valid_y, 2)
      validation_scores = model.evaluate(valid_x, evaluate_y, verbose=1)
      for name, score in zip(model.metrics_names, validation_scores):
          print(f'{name} : {score}')
      predict_y = model.predict(valid_x)
      predict_y_argmax = np.argmax(predict_y, axis=1)
      evaluate_y_argmax = np.argmax(evaluate_y, axis=1)
      cm = confusion_matrix(y_true=evaluate_y_argmax, y_pred=predict_y_argmax).ravel()
     
      tn, fp, fn, tp = cm
      class_wise_f1 = np.round(f1_score(y_true=evaluate_y_argmax, y_pred=predict_y_argmax, average=None)*100)*0.01 #from TA
      print('the mean-f1 score: {:.4f}'.format(np.mean(class_wise_f1))) 
      print(f' true negative: {tn}')
      print(f' false positive: {fp}')
      print(f' false negative: {fn}')
      print(f' true positive: {tp}')
      print(f' Total number of validation: {tn+fp+fn+tp}')
      precision_p = tp/(tp+fp)
      recall_p = tp/(tp+fn)
      precision_n = tn/(tn+fn)
      recall_n = tn/(tn+fp)
      accuracy += (tn+tp)/(tn+tp+fn+fp)
      f1_avg += np.mean(class_wise_f1)
      f1_score_p += 2*(precision_p * recall_p)/(precision_p + recall_p)
      f1_score_n += 2*(precision_n * recall_n)/(precision_n + recall_n)
      acc_p += tp/(tp+fp)
      acc_n += tn/(tn+fn)
  print(f'avg f1_score : {f1_avg/num} , p class f1_score : {f1_score_p/num}, n class f1_score : {f1_score_n/num} \n avg accuracy : {accuracy/num}, p class accuracy :{acc_p/num}, n class accuracy : {acc_n/num}' )
  # return np.mean(class_wise_f1), f1_score_p, f1_score_n, accuracy
  


def evaluation(model, valid_x, valid_y):
    predict_y = model.predict(valid_x)
    predict_y_argmax = np.argmax(predict_y, axis=1)
    evaluate_y_argmax = np.argmax(valid_y, axis=1)
    # cm = confusion_matrix(y_true=evaluate_y_argmax, y_pred=predict_y_argmax).ravel()
    tn, fp, fn, tp =confusion_matrix(y_true=evaluate_y_argmax, y_pred=predict_y_argmax).ravel()
    print(f'true negative: {tn}')
    print(f'false positive: {fp}')
    print(f'false negative: {fn}')
    print(f'true positive: {tp}')
    print(f'Total number of validation: {tn+fp+fn+tp}')
    print()
    class_wise_f1 = np.round(f1_score(y_true=evaluate_y_argmax, y_pred=predict_y_argmax, average=None)*100)*0.01 #from TA
    print('the mean-f1 score: {:.4f}'.format(np.mean(class_wise_f1)))

In [0]:
#@title Split data 

#without pre-processing
print('without processing')
samples_with_controlled  = [loadmat(os.path.join('./train', name))['BD'] for name in os.listdir('./train')]
samples_with_controlled = np.concatenate(samples_with_controlled)
all_train_x = samples_with_controlled[:,:-3]
all_train_y = samples_with_controlled[:,-1]
print(samples_with_controlled.shape)
print(all_train_x.shape, all_train_y.shape)

print('without truncating')
samples_without_truncating = np.vstack(samples)
wt_train_x = samples_without_truncating[:,:-3]
wt_train_y = samples_without_truncating[:,-1]
print(samples_without_truncating.shape)
print(wt_train_x.shape, wt_train_y.shape)

train_x = train_data[:,:-3]  #(n, 30)
train_y = train_data[:,-1]   #(n, 1)
valid_x = val_data[:,:-3]    #(n,30)
valid_y = val_data[:,-1]     #(n, 1)
print('with pre-processing')
print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)



without processing
(356107, 33)
(356107, 30) (356107,)
without truncating
(270580, 33)
(270580, 30) (270580,)
with pre-processing
(204738, 30) (204738,)
(158438, 30) (158438,)


In [0]:
#@title Split, normalise and standardise different modalities
from sklearn.preprocessing import MinMaxScaler
train_x_EngAng, valid_x_EngAng = train_x[:, :26], valid_x[:, :26]
train_x_EngSeg, valid_x_EngSeg = train_x[:, 13:], valid_x[:, 13:]
train_x_AngSeg, valid_x_AngSeg = np.concatenate((train_x[:, 0:13], train_x[:, 26:]), axis=1), \
                                 np.concatenate((valid_x[:, 0:13], valid_x[:, 26:]), axis=1)
train_x_Ang, valid_x_Ang =  train_x[:, :13], valid_x[:,:13]
train_x_Eng, valid_x_Eng = train_x[:, 13:26], valid_x[:,13:26]
train_x_sEMG, valid_x_sEMG = train_x[:, 26:], valid_x[:,26:]



minMax_Scaler = MinMaxScaler()
normalized_train_x = minMax_Scaler.fit_transform(train_x)
normalized_valid_x = minMax_Scaler.fit_transform(valid_x)

normalized_train_x_EngAng = minMax_Scaler.fit_transform(train_x_EngAng) 
normalized_valid_x_EngAng = minMax_Scaler.fit_transform(valid_x_EngAng) 

normalized_train_x_EngSeg = minMax_Scaler.fit_transform(train_x_EngSeg) 
normalized_valid_x_EngSeg = minMax_Scaler.fit_transform(valid_x_EngSeg) 

normalized_train_x_AngSeg = minMax_Scaler.fit_transform(train_x_AngSeg) 
normalized_valid_x_AngSeg = minMax_Scaler.fit_transform(valid_x_AngSeg) 

standard_scaler = StandardScaler()    ## Important -> PCA requires unit variance
standardized_train_x = standard_scaler.fit_transform(train_x)
standardized_valid_x = standard_scaler.fit_transform(valid_x)

standardized_train_x_EngAng = standard_scaler.fit_transform(train_x_EngAng) 
standardized_valid_x_EngAng = standard_scaler.fit_transform(valid_x_EngAng) 

standardized_train_x_EngSeg = standard_scaler.fit_transform(train_x_EngSeg) 
standardized_valid_x_EngSeg = standard_scaler.fit_transform(valid_x_EngSeg) 

standardized_train_x_AngSeg = standard_scaler.fit_transform(train_x_AngSeg) 
standardized_valid_x_AngSeg = standard_scaler.fit_transform(valid_x_AngSeg) 

standardized_train_x_Eng = standard_scaler.fit_transform(train_x[:,13:26]) 
standardized_valid_x_Eng = standard_scaler.fit_transform(valid_x[:,13:26]) 

standardized_train_x_Seg = standard_scaler.fit_transform(train_x[:,26:]) 
standardized_valid_x_Seg = standard_scaler.fit_transform(valid_x[:,26:]) 

standardized_train_x_Ang = standard_scaler.fit_transform(train_x[:,:13]) 
standardized_valid_x_Ang = standard_scaler.fit_transform(valid_x[:,:13]) 





##**Baseline Performance**

In [0]:
minMax_Scaler = MinMaxScaler()
main( minMax_Scaler.fit_transform(all_train_x), all_train_y, normalized_valid_x, valid_y)


================RUN 0=====================

(7733, 180, 30) (7733,)
(3340, 180, 30) (3340,)
Train on 6186 samples, validate on 1547 samples
Epoch 1/10
6186/6186 [==============================] - 207s 33ms/step - loss: 0.5244 - accuracy: 0.7886 - val_loss: 0.3823 - val_accuracy: 0.8765

Epoch 00001: val_loss improved from inf to 0.38228, saving model to stack_bilstm.h5
Epoch 2/10
6186/6186 [==============================] - 207s 33ms/step - loss: 0.4888 - accuracy: 0.7949 - val_loss: 0.3580 - val_accuracy: 0.8765

Epoch 00002: val_loss improved from 0.38228 to 0.35796, saving model to stack_bilstm.h5
Epoch 3/10
6186/6186 [==============================] - 207s 33ms/step - loss: 0.4642 - accuracy: 0.7949 - val_loss: 0.3450 - val_accuracy: 0.8785

Epoch 00003: val_loss improved from 0.35796 to 0.34497, saving model to stack_bilstm.h5
Epoch 4/10
6186/6186 [==============================] - 205s 33ms/step - loss: 0.4583 - accuracy: 0.7913 - val_loss: 0.3503 - val_accuracy: 0.8765

Epoch 00

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


In [0]:
main( minMax_Scaler.fit_transform(all_train_x[:, :26]), all_train_y, normalized_valid_x_EngAng, valid_y)

================RUN 0=====================

(7733, 180, 26) (7733,)
(3340, 180, 26) (3340,)
Train on 6186 samples, validate on 1547 samples
Epoch 1/10
6186/6186 [==============================] - 209s 34ms/step - loss: 0.5223 - accuracy: 0.7934 - val_loss: 0.4099 - val_accuracy: 0.8765

Epoch 00001: val_loss improved from inf to 0.40992, saving model to stack_bilstm.h5
Epoch 2/10
6186/6186 [==============================] - 208s 34ms/step - loss: 0.5116 - accuracy: 0.7950 - val_loss: 0.3954 - val_accuracy: 0.8765

Epoch 00002: val_loss improved from 0.40992 to 0.39539, saving model to stack_bilstm.h5
Epoch 3/10
6186/6186 [==============================] - 207s 34ms/step - loss: 0.5101 - accuracy: 0.7950 - val_loss: 0.4017 - val_accuracy: 0.8765

Epoch 00003: val_loss did not improve from 0.39539
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.335789871412123
accuracy : 0.935029923915863
the mean-f1 score: 0.4850
 true negative: 3123
 false 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


In [0]:
baseline_angseg = np.concatenate((all_train_x[:, 0:13], all_train_x[:, 26:]), axis=1)
main( minMax_Scaler.fit_transform(baseline_angseg), all_train_y, normalized_valid_x_AngSeg, valid_y)

================RUN 0=====================

(7733, 180, 17) (7733,)
(3340, 180, 17) (3340,)
Train on 6186 samples, validate on 1547 samples
Epoch 1/10
6186/6186 [==============================] - 209s 34ms/step - loss: 0.5166 - accuracy: 0.7932 - val_loss: 0.3697 - val_accuracy: 0.8765

Epoch 00001: val_loss improved from inf to 0.36968, saving model to stack_bilstm.h5
Epoch 2/10
6186/6186 [==============================] - 208s 34ms/step - loss: 0.4952 - accuracy: 0.7957 - val_loss: 0.3649 - val_accuracy: 0.8798

Epoch 00002: val_loss improved from 0.36968 to 0.36490, saving model to stack_bilstm.h5
Epoch 3/10
6186/6186 [==============================] - 208s 34ms/step - loss: 0.4729 - accuracy: 0.7978 - val_loss: 0.3679 - val_accuracy: 0.8765

Epoch 00003: val_loss did not improve from 0.36490
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.3386048440358596
accuracy : 0.9329341053962708
the mean-f1 score: 0.4900
 true negative: 3115
 fals

In [0]:
main( minMax_Scaler.fit_transform(all_train_x[:, 13:]), all_train_y, normalized_valid_x_EngSeg, valid_y)


================RUN 0=====================

(7733, 180, 17) (7733,)
(3340, 180, 17) (3340,)
Train on 6186 samples, validate on 1547 samples
Epoch 1/10
6186/6186 [==============================] - 209s 34ms/step - loss: 0.5273 - accuracy: 0.7962 - val_loss: 0.4006 - val_accuracy: 0.8772

Epoch 00001: val_loss improved from inf to 0.40063, saving model to stack_bilstm.h5
Epoch 2/10
6186/6186 [==============================] - 208s 34ms/step - loss: 0.4947 - accuracy: 0.7960 - val_loss: 0.3972 - val_accuracy: 0.8746

Epoch 00002: val_loss improved from 0.40063 to 0.39723, saving model to stack_bilstm.h5
Epoch 3/10
6186/6186 [==============================] - 208s 34ms/step - loss: 0.5073 - accuracy: 0.7978 - val_loss: 0.3804 - val_accuracy: 0.8765

Epoch 00003: val_loss improved from 0.39723 to 0.38042, saving model to stack_bilstm.h5
Epoch 4/10
6186/6186 [==============================] - 206s 33ms/step - loss: 0.5036 - accuracy: 0.7965 - val_loss: 0.3980 - val_accuracy: 0.8765

Epoch 00

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars


#Experiment with different feature selection and modalities

## Without feature selection


### All Modulality

In [0]:
main(normalized_train_x, train_y, normalized_valid_x, valid_y)

================RUN 0=====================

(4369, 180, 30) (4369,)
(3340, 180, 30) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6547 - accuracy: 0.6386 - val_loss: 0.5264 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.52639, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6450 - accuracy: 0.6406 - val_loss: 0.5658 - val_accuracy: 0.7941

Epoch 00002: val_loss did not improve from 0.52639
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.513061963334055
accuracy : 0.8269460797309875
the mean-f1 score: 0.5200
 true negative: 2716
 false positive: 407
 false negative: 171
 true positive: 46
 Total number of validation: 3340
avg f1_score : 0.52 , p class f1_score : 0.1373134328358209, n class f1_score : 0.9038269550748752 
 avg accuracy : 0.8269461077844311, p class accu

### Joint Angles and Energies

In [0]:
main(normalized_train_x_EngAng, train_y, normalized_valid_x_EngAng, valid_y)

================RUN 0=====================

(4369, 180, 26) (4369,)
(3340, 180, 26) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6610 - accuracy: 0.6332 - val_loss: 0.6015 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.60154, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6567 - accuracy: 0.6366 - val_loss: 0.5418 - val_accuracy: 0.7986

Epoch 00002: val_loss improved from 0.60154 to 0.54184, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6576 - accuracy: 0.6358 - val_loss: 0.5658 - val_accuracy: 0.7975

Epoch 00003: val_loss did not improve from 0.54184
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.49387255996287227
accuracy : 0.9290419220924377
the mean-f1 score: 0.5000
 true negative: 3098
 fals

### Joint Angles and Seng

In [0]:
main(normalized_train_x_AngSeg, train_y, normalized_valid_x_AngSeg, valid_y)

================RUN 0=====================

(4369, 180, 17) (4369,)
(3340, 180, 17) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6584 - accuracy: 0.6335 - val_loss: 0.5923 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.59235, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6325 - accuracy: 0.6567 - val_loss: 0.5238 - val_accuracy: 0.7414

Epoch 00002: val_loss improved from 0.59235 to 0.52384, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6058 - accuracy: 0.6904 - val_loss: 0.4520 - val_accuracy: 0.7540

Epoch 00003: val_loss improved from 0.52384 to 0.45197, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.5973 - accuracy: 0.7010 - val_loss: 0.4757 - val_accuracy: 0.7506

Epoch 000

### Joint Engergies and Seng

In [0]:
main(normalized_train_x_EngSeg, train_y, normalized_valid_x_EngSeg, valid_y)

================RUN 0=====================

(4369, 180, 17) (4369,)
(3340, 180, 17) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6518 - accuracy: 0.6498 - val_loss: 0.5944 - val_accuracy: 0.6785

Epoch 00001: val_loss improved from inf to 0.59438, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6427 - accuracy: 0.6609 - val_loss: 0.5725 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.59438 to 0.57246, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6579 - accuracy: 0.6378 - val_loss: 0.5632 - val_accuracy: 0.7929

Epoch 00003: val_loss improved from 0.57246 to 0.56321, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 117s 33ms/step - loss: 0.6614 - accuracy: 0.6343 - val_loss: 0.5686 - val_accuracy: 0.7929

Epoch 000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


## PCA Run

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
#Feature selection models
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Feature Selection with Univariate Statistical Tests
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [0]:
def pca_selection(data):
    pca = PCA(n_components =0.99)
    reduced = pca.fit_transform(data)
    print("number of features selected: {}".format(pca.n_components_))
    # print(pca.n_components)
    # print(pca.components_)
    # print(pca.explained_variance_)
    # print(pca.n_features_)

    return reduced

### All modulality

In [0]:
pca_train_x = pca_selection(standardized_train_x)
pca_valid_x = pca_selection(standardized_valid_x)
main(pca_train_x, train_y, pca_valid_x, valid_y)

number of features selected: 26
number of features selected: 26
================RUN 0=====================

(4369, 180, 26) (4369,)
(3340, 180, 26) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.5695 - accuracy: 0.7070 - val_loss: 0.4846 - val_accuracy: 0.7998

Epoch 00001: val_loss improved from inf to 0.48461, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 117s 33ms/step - loss: 0.4655 - accuracy: 0.7877 - val_loss: 0.4908 - val_accuracy: 0.8169

Epoch 00002: val_loss did not improve from 0.48461
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.6556438806885017
accuracy : 0.6371257305145264
the mean-f1 score: 0.4600
 true negative: 2025
 false positive: 1098
 false negative: 114
 true positive: 103
 Total number of validation: 3340
avg f1_score : 0.46 , p class f1_score : 0.14527503526093088, n class f1_score : 

### Joint Angles and Energies

In [0]:
pca_train_x = pca_selection(standardized_train_x_EngAng)
pca_valid_x = pca_selection(standardized_valid_x_EngAng)
print(pca_train_x.shape, train_y.shape, pca_valid_x.shape, valid_y.shape)

main(pca_train_x, train_y, pca_valid_x, valid_y)

number of features selected: 22
number of features selected: 22
(204738, 22) (204738,) (158438, 22) (158438,)
================RUN 0=====================

(4369, 180, 22) (4369,)
(3340, 180, 22) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6076 - accuracy: 0.6446 - val_loss: 0.4395 - val_accuracy: 0.8101

Epoch 00001: val_loss improved from inf to 0.43952, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.4824 - accuracy: 0.7685 - val_loss: 0.4745 - val_accuracy: 0.7700

Epoch 00002: val_loss did not improve from 0.43952
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.5677563190103291
accuracy : 0.6649700403213501
the mean-f1 score: 0.4450
 true negative: 2157
 false positive: 966
 false negative: 153
 true positive: 64
 Total number of validation: 3340
avg f1_score : 0.445 , p class f1_sco

###Joint Angles and Seng

In [0]:
pca_train_x = pca_selection(standardized_train_x_AngSeg )
pca_valid_x = pca_selection(standardized_valid_x_AngSeg)
print(pca_train_x.shape, train_y.shape, pca_valid_x.shape, valid_y.shape)
main(pca_train_x, train_y, pca_valid_x, valid_y)

number of features selected: 14
number of features selected: 14
(204738, 14) (204738,) (158438, 14) (158438,)
================RUN 0=====================

(4369, 180, 14) (4369,)
(3340, 180, 14) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6078 - accuracy: 0.6764 - val_loss: 0.5058 - val_accuracy: 0.7677

Epoch 00001: val_loss improved from inf to 0.50584, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.4685 - accuracy: 0.7906 - val_loss: 0.4942 - val_accuracy: 0.7666

Epoch 00002: val_loss improved from 0.50584 to 0.49424, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.4093 - accuracy: 0.8312 - val_loss: 0.5774 - val_accuracy: 0.6888

Epoch 00003: val_loss did not improve from 0.49424
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step

###Joint Energies and Seng

In [0]:
pca_train_x = pca_selection(standardized_train_x_EngSeg)
pca_valid_x = pca_selection(standardized_valid_x_EngSeg)
print(pca_train_x.shape, train_y.shape, pca_valid_x.shape, valid_y.shape)
main(pca_train_x, train_y, pca_valid_x, valid_y)

number of features selected: 17
number of features selected: 17
(204738, 17) (204738,) (158438, 17) (158438,)
================RUN 0=====================

(4369, 180, 17) (4369,)
(3340, 180, 17) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6037 - accuracy: 0.6715 - val_loss: 0.4878 - val_accuracy: 0.7769

Epoch 00001: val_loss improved from inf to 0.48784, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.5483 - accuracy: 0.7325 - val_loss: 0.5143 - val_accuracy: 0.7586

Epoch 00002: val_loss did not improve from 0.48784
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.6849631640725506
accuracy : 0.6050898432731628
the mean-f1 score: 0.4600
 true negative: 1881
 false positive: 1242
 false negative: 77
 true positive: 140
 Total number of validation: 3340
avg f1_score : 0.45999999999999996 ,

## ANOVA Run

In [0]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [0]:
def anova(X, Y, k=10):
    test = SelectKBest(score_func=f_classif, k=k)
    fit = test.fit(X, Y)
    support = test.get_support()
    # summarize scores
    set_printoptions(precision=3)
    features = fit.transform(X)
    # summarize selected features
    print(features.shape)
    return features, support

### All Modality

In [0]:
for i in range(6,31):
  anova_train_x, train_support = anova(normalized_train_x, train_y, i)
  print(train_support)
  anova_valid_x = normalized_valid_x[:][:, train_support]
  main(anova_train_x, train_y, anova_valid_x, valid_y)

(204738, 6)
[False False False False False False False False False False False False
 False False False  True False  True False False False False False False
 False False  True  True  True  True]
================RUN 0=====================

(4369, 180, 6) (4369,)
(3340, 180, 6) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6578 - accuracy: 0.6469 - val_loss: 0.6202 - val_accuracy: 0.6476

Epoch 00001: val_loss improved from inf to 0.62016, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6326 - accuracy: 0.6601 - val_loss: 0.5735 - val_accuracy: 0.6579

Epoch 00002: val_loss improved from 0.62016 to 0.57353, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6131 - accuracy: 0.6784 - val_loss: 0.6098 - val_accuracy: 0.6636

Epoch 00003: val_loss did not improve from 0.57353


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 9) (4369,)
(3340, 180, 9) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6562 - accuracy: 0.6335 - val_loss: 0.6240 - val_accuracy: 0.7414

Epoch 00001: val_loss improved from inf to 0.62402, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6327 - accuracy: 0.6712 - val_loss: 0.5482 - val_accuracy: 0.7597

Epoch 00002: val_loss improved from 0.62402 to 0.54816, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6312 - accuracy: 0.6704 - val_loss: 0.5362 - val_accuracy: 0.7677

Epoch 00003: val_loss improved from 0.54816 to 0.53615, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6258 - accuracy: 0.6790 - val_loss: 0.5531 - val_accuracy: 0.7735

Epoch 00004: val_loss did not improve from 0.53615
Epoc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 12) (4369,)
(3340, 180, 12) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6573 - accuracy: 0.6340 - val_loss: 0.5658 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56585, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6381 - accuracy: 0.6601 - val_loss: 0.5485 - val_accuracy: 0.7975

Epoch 00002: val_loss improved from 0.56585 to 0.54846, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6523 - accuracy: 0.6421 - val_loss: 0.5567 - val_accuracy: 0.7929

Epoch 00003: val_loss did not improve from 0.54846
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.4742956421332445
accuracy : 0.9335329532623291
the mean-f1 score: 0.5350
 true negative: 3106
 false positive: 17
 false negative: 205
 true pos

...

(4369, 180, 16) (4369,)
(3340, 180, 16) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6601 - accuracy: 0.6315 - val_loss: 0.5678 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56783, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6409 - accuracy: 0.6584 - val_loss: 0.5678 - val_accuracy: 0.7918

Epoch 00002: val_loss improved from 0.56783 to 0.56780, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6371 - accuracy: 0.6538 - val_loss: 0.5497 - val_accuracy: 0.7620

Epoch 00003: val_loss improved from 0.56780 to 0.54966, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 117s 33ms/step - loss: 0.6297 - accuracy: 0.6712 - val_loss: 0.8296 - val_accuracy: 0.2449

Epoch 00004: val_loss did not improve from 0.54966
Ep

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 21) (4369,)
(3340, 180, 21) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6620 - accuracy: 0.6338 - val_loss: 0.5565 - val_accuracy: 0.7517

Epoch 00001: val_loss improved from inf to 0.55648, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6576 - accuracy: 0.6361 - val_loss: 0.5610 - val_accuracy: 0.7929

Epoch 00002: val_loss did not improve from 0.55648
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.4732169276939895
accuracy : 0.935029923915863
the mean-f1 score: 0.4850
 true negative: 3123
 false positive: 0
 false negative: 217
 true positive: 0
 Total number of validation: 3340
avg f1_score : 0.485 , p class f1_score : nan, n class f1_score : 0.9664242611790191 
 avg accuracy : 0.9350299401197605, p class accuracy :nan, n class accuracy : 0.9350299401197605
(204738, 22)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 22) (4369,)
(3340, 180, 22) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6637 - accuracy: 0.6235 - val_loss: 0.5927 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.59270, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6379 - accuracy: 0.6538 - val_loss: 0.5321 - val_accuracy: 0.7906

Epoch 00002: val_loss improved from 0.59270 to 0.53211, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6254 - accuracy: 0.6775 - val_loss: 0.5545 - val_accuracy: 0.7563

Epoch 00003: val_loss did not improve from 0.53211
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.6147813997582761
accuracy : 0.67514967918396
the mean-f1 score: 0.5200
 true negative: 2073
 false positive: 1050
 false negative: 35
 true posi

### Joint Angles and Energies

In [0]:
#14
for i in range(8,25):
  anova_train_x, train_support = anova(normalized_train_x_EngAng , train_y , i)
  anova_valid_x = normalized_valid_x_EngAng[:][:, train_support]
  print(train_support)
  main(anova_train_x, train_y, anova_valid_x, valid_y)

(204738, 8)
[ True  True False False False False False False False  True False False
 False  True False  True  True  True  True False False False False False
 False False]
================RUN 0=====================

(4369, 180, 8) (4369,)
(3340, 180, 8) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6600 - accuracy: 0.6283 - val_loss: 0.5970 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.59704, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 117s 33ms/step - loss: 0.6574 - accuracy: 0.6343 - val_loss: 0.5570 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.59704 to 0.55700, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 116s 33ms/step - loss: 0.6571 - accuracy: 0.6343 - val_loss: 0.5949 - val_accuracy: 0.7929

Epoch 00003: val_loss did not improve from 0.55700
Epoch 00003: early stopp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 10) (4369,)
(3340, 180, 10) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6620 - accuracy: 0.6338 - val_loss: 0.5656 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56558, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6581 - accuracy: 0.6343 - val_loss: 0.5511 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.56558 to 0.55106, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6568 - accuracy: 0.6343 - val_loss: 0.5861 - val_accuracy: 0.7929

Epoch 00003: val_loss did not improve from 0.55106
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.5197583963057238
accuracy : 0.9347305297851562
the mean-f1 score: 0.4850
 true negative: 3122
 false positive: 1
 false negative: 217
 true posi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars


(4369, 180, 11) (4369,)
(3340, 180, 11) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6615 - accuracy: 0.6283 - val_loss: 0.5941 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.59407, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6581 - accuracy: 0.6343 - val_loss: 0.5847 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.59407 to 0.58474, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6541 - accuracy: 0.6343 - val_loss: 0.5606 - val_accuracy: 0.7929

Epoch 00003: val_loss improved from 0.58474 to 0.56057, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6599 - accuracy: 0.6260 - val_loss: 0.5846 - val_accuracy: 0.7929

Epoch 00004: val_loss did not improve from 0.56057
Ep

...

(4369, 180, 21) (4369,)
(3340, 180, 21) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6603 - accuracy: 0.6338 - val_loss: 0.5636 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56360, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6580 - accuracy: 0.6318 - val_loss: 0.5813 - val_accuracy: 0.7929

Epoch 00002: val_loss did not improve from 0.56360
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.505010641656236
accuracy : 0.935029923915863
the mean-f1 score: 0.4850
 true negative: 3123
 false positive: 0
 false negative: 217
 true positive: 0
 Total number of validation: 3340
avg f1_score : 0.485 , p class f1_score : nan, n class f1_score : 0.9664242611790191 
 avg accuracy : 0.9350299401197605, p class accuracy :nan, n class accuracy : 0.9350299401197605
(204738, 22)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 22) (4369,)
(3340, 180, 22) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6656 - accuracy: 0.6203 - val_loss: 0.5899 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.58992, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6584 - accuracy: 0.6340 - val_loss: 0.5677 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.58992 to 0.56775, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6502 - accuracy: 0.6358 - val_loss: 0.5611 - val_accuracy: 0.7918

Epoch 00003: val_loss improved from 0.56775 to 0.56112, saving model to stack_bilstm.h5
Epoch 4/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6419 - accuracy: 0.6455 - val_loss: 0.5295 - val_accuracy: 0.7288

Epoch 00004: val_loss improved from 0.56112 to 0.5294

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


(4369, 180, 24) (4369,)
(3340, 180, 24) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6574 - accuracy: 0.6346 - val_loss: 0.5634 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56345, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6531 - accuracy: 0.6378 - val_loss: 0.5805 - val_accuracy: 0.7975

Epoch 00002: val_loss did not improve from 0.56345
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.5179250191785618
accuracy : 0.9203592538833618
the mean-f1 score: 0.4850
 true negative: 3073
 false positive: 50
 false negative: 216
 true positive: 1
 Total number of validation: 3340
avg f1_score : 0.485 , p class f1_score : 0.007462686567164179, n class f1_score : 0.9585152838427947 
 avg accuracy : 0.9203592814371258, p class accuracy :0.0196078431372549, n class accuracy

###Joint Angles and Seng

In [0]:
for i in range(6,15):
  anova_train_x, train_support = anova(normalized_train_x_AngSeg , train_y, i)
  anova_valid_x = normalized_valid_x_AngSeg[:][:, train_support]
  print(train_support)
  main(anova_train_x, train_y, anova_valid_x, valid_y)

(204738, 6)
[ True  True False False False False False False False False False False
 False  True  True  True  True]
================RUN 0=====================

(4369, 180, 6) (4369,)
(3340, 180, 6) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6598 - accuracy: 0.6309 - val_loss: 0.5348 - val_accuracy: 0.7963

Epoch 00001: val_loss improved from inf to 0.53483, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6396 - accuracy: 0.6598 - val_loss: 0.5090 - val_accuracy: 0.7975

Epoch 00002: val_loss improved from 0.53483 to 0.50899, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6477 - accuracy: 0.6443 - val_loss: 0.5730 - val_accuracy: 0.7975

Epoch 00003: val_loss did not improve from 0.50899
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars


###Joint Energies and Seng

In [0]:
for i in range(6,15):

  anova_train_x, train_support = anova(normalized_train_x_EngSeg , train_y, i )
  anova_valid_x = normalized_valid_x_EngSeg[:][:, train_support]
  print(train_support)
  main(anova_train_x, train_y, anova_valid_x, valid_y)


(204738, 6)
[False False  True False  True False False False False False False False
 False  True  True  True  True]
================RUN 0=====================

(4369, 180, 6) (4369,)
(3340, 180, 6) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6623 - accuracy: 0.6358 - val_loss: 0.5972 - val_accuracy: 0.7712

Epoch 00001: val_loss improved from inf to 0.59724, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6443 - accuracy: 0.6472 - val_loss: 0.5208 - val_accuracy: 0.7643

Epoch 00002: val_loss improved from 0.59724 to 0.52078, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6332 - accuracy: 0.6647 - val_loss: 0.5326 - val_accuracy: 0.7643

Epoch 00003: val_loss did not improve from 0.52078
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms

##Random Forest Run


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


def forest_selection(X, y, estimator=30):
    y = np.ravel(y)
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=estimator))
    embeded_rf_selector.fit(X, y)
    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature = embeded_rf_selector.transform(X)
    # print('selected features {}'.format(embeded_rf_feature.shape))
    return embeded_rf_feature, embeded_rf_support
  



###All Modality


In [0]:
rf_train_x, train_support = forest_selection(normalized_train_x, train_y)
print(train_support)
rf_valid_x = normalized_valid_x[:][:, train_support]
main(rf_train_x, train_y, rf_valid_x, valid_y)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False False False  True False]
================RUN 0=====================

(4369, 180, 14) (4369,)
(3340, 180, 14) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6627 - accuracy: 0.6298 - val_loss: 0.5816 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.58155, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6592 - accuracy: 0.6320 - val_loss: 0.5677 - val_accuracy: 0.7929

Epoch 00002: val_loss improved from 0.58155 to 0.56775, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6549 - accuracy: 0.6349 - val_loss: 0.5243 - val_accuracy: 0.7929

Epoch 00003: val_loss improved from 0.56775 to 0.52431, savin

###Joint Angles and Energies

In [0]:
rf_train_x, train_support = forest_selection(normalized_train_x_EngAng, train_y)
print(train_support)
rf_valid_x = normalized_valid_x_EngAng[:][:, train_support]
main(rf_train_x, train_y, rf_valid_x, valid_y)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False False False False False False
 False False]
================RUN 0=====================

(4369, 180, 13) (4369,)
(3340, 180, 13) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6606 - accuracy: 0.6320 - val_loss: 0.5673 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.56732, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6593 - accuracy: 0.6343 - val_loss: 0.6009 - val_accuracy: 0.7929

Epoch 00002: val_loss did not improve from 0.56732
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.5489154868497106
accuracy : 0.935029923915863
the mean-f1 score: 0.4850
 true negative: 3123
 false positive: 0
 false negative: 217
 true positive: 0
 Total number of validati

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


###Joint Angles and Seng

In [0]:
rf_train_x, train_support = forest_selection(normalized_train_x_AngSeg, train_y)
print(train_support)
rf_valid_x = normalized_valid_x_AngSeg[:][:, train_support]
main(rf_train_x, train_y, rf_valid_x, valid_y)


[ True  True  True  True  True  True  True  True  True  True  True  True
 False False False False False]
================RUN 0=====================

(4369, 180, 12) (4369,)
(3340, 180, 12) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 119s 34ms/step - loss: 0.6644 - accuracy: 0.6278 - val_loss: 0.5772 - val_accuracy: 0.7929

Epoch 00001: val_loss improved from inf to 0.57720, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6586 - accuracy: 0.6343 - val_loss: 0.5977 - val_accuracy: 0.7929

Epoch 00002: val_loss did not improve from 0.57720
Epoch 00002: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss : 0.5412983117703192
accuracy : 0.935029923915863
the mean-f1 score: 0.4850
 true negative: 3123
 false positive: 0
 false negative: 217
 true positive: 0
 Total number of validation: 3340
avg f1_score : 0.485 , p class f1_score : nan,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in long_scalars


###Joint Energies and Seng

In [0]:
rf_train_x, train_support = forest_selection(normalized_train_x_EngSeg, train_y)
print(train_support)
rf_valid_x = normalized_valid_x_EngSeg[:][:, train_support]
main(rf_train_x, train_y, rf_valid_x, valid_y)


[ True False  True  True False False False False False False False False
 False  True  True  True  True]
================RUN 0=====================

(4369, 180, 7) (4369,)
(3340, 180, 7) (3340,)
Train on 3495 samples, validate on 874 samples
Epoch 1/10
3495/3495 [==============================] - 120s 34ms/step - loss: 0.6631 - accuracy: 0.6361 - val_loss: 0.6085 - val_accuracy: 0.6522

Epoch 00001: val_loss improved from inf to 0.60854, saving model to stack_bilstm.h5
Epoch 2/10
3495/3495 [==============================] - 118s 34ms/step - loss: 0.6391 - accuracy: 0.6638 - val_loss: 0.5320 - val_accuracy: 0.8112

Epoch 00002: val_loss improved from 0.60854 to 0.53201, saving model to stack_bilstm.h5
Epoch 3/10
3495/3495 [==============================] - 117s 34ms/step - loss: 0.6505 - accuracy: 0.6515 - val_loss: 0.5324 - val_accuracy: 0.8124

Epoch 00003: val_loss did not improve from 0.53201
Epoch 00003: early stopping
3340/3340 [==============================] - 9s 3ms/step
loss :

##Lasso

In [0]:
# from sklearn.feature_selection import SelectFromModel
# from sklearn.linear_model import LogisticRegression
# def lasso_selection(X, y):
#     y = np.ravel(y)
#     embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1", solver='saga'))
#     embeded_lr_selector.fit(X, y)
#     embeded_lr_support = embeded_lr_selector.get_support()
#     embeded_lr_feature = embeded_lr_selector.transform(X)
#     # print('selected features {}'.format(embeded_lr_feature.shape))
#     return embeded_lr_feature, embeded_lr_support

In [0]:
# lf_train_x, train_support = lasso_selection(standardized_train_x, train_y)
# print(train_support)
# if not lf_train_x.shape == standardized_train_x.shape:
#   lf_valid_x = standardized_valid_x[:][:, train_support]
#   main(lf_train_x, train_y, lf_valid_x, valid_y)


In [0]:
# lf_train_x, train_support = lasso_selection(standardized_train_x_EngAng, train_y)
# print(train_support)
# if not lf_train_x.shape == standardized_train_x_EngAng.shape:
#   lf_valid_x = standardized_valid_x_EngAng[:][:, train_support]
#   main(lf_train_x, train_y, lf_valid_x, valid_y)


In [0]:
# lf_train_x, train_support = lasso_selection(standardized_train_x_EngSeg, train_y)
# print(train_support)
# if not lf_train_x.shape == standardized_train_x_EngSeg.shape:
#   lf_valid_x = standardized_valid_x_EngSeg[:][:, train_support]
#   main(lf_train_x, train_y, lf_valid_x, valid_y)


In [0]:
# lf_train_x, train_support = lasso_selection(standardized_train_x_AngSeg, train_y)
# print(train_support)
# if not lf_train_x.shape == standardized_train_x_AngSeg.shape:
#   lf_valid_x = standardized_valid_x_AngSeg[:][:, train_support]
#   main(lf_train_x, train_y, lf_valid_x, valid_y)
